# A Typescript Notebook

This book uses [typescript-jupyter-kernel](https://github.com/simplyhexagonal/typescript-jupyter-kernel) to execute Typescript code.

## Accessing the Filesystem

For example we can load some csv data from the local filesystem using `fs` and parse the data using [d3-dsv](https://github.com/d3/d3-dsv)

In [ ]:
import { readFileSync } from 'fs';

const { csvParse } = require('./d3-dsv.js');

const rawData = readFileSync('./data.csv').toString('utf-8');

const parsedData = csvParse(rawData);

jupyter.out(parsedData);

[
 {
 Country: 'Luxembourg',
 'Hourly Minimum Wage in USD (2015)': '11.9',
 'Hourly Minimum Wage in USD (2016)': '11.8',
 'Hourly Minimum Wage in USD (2017)': '12.1',
 'Hourly Minimum Wage in USD (2018)': '12',
 'Hourly Minimum Wage in USD (2019)': '12.2',
 'Hourly Minimum Wage in USD (2020)': '12.4'
 },
 {
 Country: 'Australia',
 'Hourly Minimum Wage in USD (2015)': '11.6',
 'Hourly Minimum Wage in USD (2016)': '11.8',
 'Hourly Minimum Wage in USD (2017)': '11.9',
 'Hourly Minimum Wage in USD (2018)': '12',
 'Hourly Minimum Wage in USD (2019)': '12.2',
 'Hourly Minimum Wage in USD (2020)': '12.4'
 },
 {
 Country: 'France',
 'Hourly Minimum Wage in USD (2015)': '12',
 'Hourly Minimum Wage in USD (2016)': '12.1',
 'Hourly Minimum Wage in USD (2017)': '12.1',
 'Hourly Minimum Wage in USD (2018)': '12',
 'Hourly Minimum Wage in USD (2019)': '12.1',
 'Hourly Minimum Wage in USD (2020)': '12.1'
 },
 {
 Country: 'Germany',
 'Hourly Minimum Wage in USD (2015)': '11.3',
 'Hourly Minimum Wage in USD (2016)': '11.3',
 'Hourly Minimum Wage in USD (2017)': '11.6',
 'Hourly Minimum Wage in USD (2018)': '11.4',
 'Hourly Minimum Wage in USD (2019)': '11.7',
 'Hourly Minimum Wage in USD (2020)': '11.8'
 },
 {
 Country: 'New Zealand',
 'Hourly Minimum Wage in USD (2015)': '9.8',
 'Hourly Minimum Wage in USD (2016)': '10.1',
 'Hourly Minimum Wage in USD (2017)': '10.2',
 'Hourly Minimum Wage in USD (2018)': '10.4',
 'Hourly Minimum Wage in USD (2019)': '11',
 'Hourly Minimum Wage in USD (2020)': '11.6'
 },
 columns: [
 'Country',
 'Hourly Minimum Wage in USD (2015)',
 'Hourly Minimum Wage in USD (2016)',
 'Hourly Minimum Wage in USD (2017)',
 'Hourly Minimum Wage in USD (2018)',
 'Hourly Minimum Wage in USD (2019)',
 'Hourly Minimum Wage in USD (2020)'
 ]
]

## Rendering to HTML

We can take the previous code and use the convenience `jupyter` object to render an HTML table:

In [ ]:
import { readFileSync } from 'fs';

const { csvParseRows } = require('./d3-dsv.js');

const rawData = readFileSync('./data.csv').toString('utf-8');

const [
  columnHeaders,
  ...rows
] = csvParseRows(rawData);

const table = jupyter.dom.window.document.createElement('table');
const tableHeader = jupyter.dom.window.document.createElement('thead');
const tableHeaderRow = jupyter.dom.window.document.createElement('tr');
const tableBody = jupyter.dom.window.document.createElement('tbody');

columnHeaders.forEach(
  (h) => {
    const columnHeader = jupyter.dom.window.document.createElement('th');
    columnHeader.innerHTML = h;

    tableHeaderRow.appendChild(columnHeader);
  }
);

tableHeader.appendChild(tableHeaderRow);

rows.forEach(
  (r) => {
    const rowElement = jupyter.dom.window.document.createElement('tr');

    r.forEach(
      (d) => {
        const dataElement = jupyter.dom.window.document.createElement('td');
        dataElement.innerHTML = d;

        rowElement.appendChild(dataElement);
      }
    );

    tableBody.appendChild(rowElement);
  }
);

table.appendChild(tableHeader);

table.appendChild(tableBody);

jupyter.dom.window.document.body.appendChild(table);

jupyter.render();


Country,Hourly Minimum Wage in USD (2015),Hourly Minimum Wage in USD (2016),Hourly Minimum Wage in USD (2017),Hourly Minimum Wage in USD (2018),Hourly Minimum Wage in USD (2019),Hourly Minimum Wage in USD (2020)
Luxembourg,11.9,11.8,12.1,12,12.2,12.4
Australia,11.6,11.8,11.9,12,12.2,12.4
France,12,12.1,12.1,12,12.1,12.1
Germany,11.3,11.3,11.6,11.4,11.7,11.8
New Zealand,9.8,10.1,10.2,10.4,11,11.6


There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)